# Test plane extraction for a single point cloud

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
import os
import time
import open3d as o3d
import plotly.graph_objects as go

import planeslam.general as general
from planeslam.mesh import LidarMesh
from planeslam.scan import Scan
from planeslam.clustering import cluster_mesh_graph_search, plot_clusters
from planeslam.extraction import scan_from_clusters
import planeslam.io as io

%load_ext autoreload
%autoreload 2
%load_ext line_profiler

Read Point Cloud from pcd file

In [ ]:
binpath = os.path.join(os.getcwd(),'..', 'data', 'airsim', 'blocks_10_samples_5.0hz', 'lidar', 'Drone0')
PCs = io.read_lidar_bin(binpath)
P = PCs[0]

In [ ]:
# Convert points to ENU
P = general.NED_to_ENU(P)

# Downsample the points
P = general.downsample(P, factor=2, axis=0)

In [ ]:
# Plot the points 
fig = go.Figure(data=general.pc_plot_trace(P))
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

Cluster the points

In [ ]:
# Create the mesh
start_time = time.time()
mesh = LidarMesh(P)
print("elapsed time: ", time.time() - start_time)

In [ ]:
start_time = time.time()
mesh.prune(edge_len_lim=10)
print("elapsed time: ", time.time() - start_time)

In [ ]:
# Cluster the mesh with graph search
start_time = time.time()
clusters, avg_normals = cluster_mesh_graph_search(mesh)
print("elapsed time: ", time.time() - start_time)

In [ ]:
#%lprun -f cluster_mesh_graph_search cluster_mesh_graph_search(mesh)

In [ ]:
# Plot mesh
fig = go.Figure(data=mesh.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Plot clusters
plot_clusters(P, mesh, clusters)

Extract planes

In [ ]:
planes, vertices, faces = scan_from_clusters(mesh, clusters, avg_normals)
scan = Scan(planes, vertices, faces)

In [ ]:
# Plot the scan
fig = go.Figure(data=scan.plot_trace())
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()